In [ ]:
using Plots, ColorSchemes, SparseArrays

In [ ]:
function ∫(f,a,b;n=10000)
    h=BigFloat((b-a)/n) 
    x=BigFloat(0)
    for i ∈ 1:n
       x+=(h*f(a+i*h))
    end
    return x
end

function coef_fourier(f,L,N)
    coeficientes = []
    for i ∈ 1:N
        g(x) = f(x)*sin(i*π*x/L)
        x = (2/L)*∫(g,0,L)
        push!(coeficientes,x)
    end
    return coeficientes
end

function aprox_fourier(coeficientes,x,L;T₁=0,T₂=0)
    suma = 0
    for i ∈ 1:length(coeficientes)
        suma += coeficientes[i]*sin(i*π*x/L)
    end
    suma += ((T₂-T₁)/L)*x + T₁
    return suma
end

function coef_fourier_general(f,L;T₁=0,T₂=0,N=50)
    g(x) = f(x) - ((T₂-T₁)/L)*x - T₁
    coeficientes = coef_fourier(g,L,N) 
    return coeficientes
end

function solucion_cal_dirichlet(x,t,coeficientes,L;κ=0.01,T₁=0,T₂=0)
    suma = 0
    for i ∈ 1:length(coeficientes)
        suma += coeficientes[i]*sin(i*π*x/L)*exp(-((i*π/L)^2)*t*κ)
    end
    suma += ((T₂-T₁)/L)*x + T₁
    return suma
end

In [ ]:
f(x) = -x^2 + 2*x
xs = 0:0.001:2
coeficientes = coef_fourier_general(f,2)

In [ ]:
animacion = @animate for t ∈ 0:0.1:100
    ys = []
    for i ∈ 1:length(xs)
        y = solucion_cal_dirichlet(xs[i],t,coeficientes,2)
        push!(ys,y)
    end
    plot(xs,ys,line_z=ys,linecolor=:thermal,linewidth=3,label=false,title ="t = $(t) s",
        ylims=(-0.1,1.02),xlims=(0,2),clim =(0,1))
    plot!(xs,-0.1*ones(length(xs)),line_z=ys,linecolor=:thermal,linewidth=40,label=false,dpi=150)
end

In [ ]:
gif(animacion,"sol_1.gif",fps =20)

In [ ]:
x2 = 0:0.001:4
h(x) = 4*x*(x-1)*(x-3)^2+7
coeficientes2 = coef_fourier_general(h,4;T₁=7,T₂=55,N=100)

In [ ]:
animacion = @animate for t ∈ 0:0.1:100
    y2 = []
    for i ∈ 1:length(x2)
        y = solucion_cal_dirichlet(x2[i],t,coeficientes2,4;T₁=7,T₂=55)
        push!(y2,y)
    end
    plot(x2,y2,line_z=y2,linecolor=:thermal,linewidth=5,label=false,title="t = $(t) s",
        ylims=(-5,55),xlims=(0,4),clim = (7,55))
    plot!(x2,-5*ones(length(x2)),line_z=y2,linecolor=:thermal,linewidth=40,label=false,dpi=150)
end

In [ ]:
gif(animacion, "sol_2.gif", fps =20)

In [ ]:
function coef_fourier_cos(f,L,N)
    coeficientes = []
    for i ∈ 0:N
        g(x) = f(x)*cos(i*π*x/L)
        x = (2/L)*∫(g,0,L)
        push!(coeficientes,x)
    end
    return coeficientes
end

function solucion_cal_neumann(x,t,coeficientes,L;κ=0.01)
    suma = coeficientes[1]/2
    for i ∈ 2:length(coeficientes)
        suma += coeficientes[i]*cos((i-1)*π*x/L)*exp(-(((i-1)*π/L)^2)*t*κ)
    end
    return suma
end

In [ ]:
f(x) = -x^2 + 2*x
xs = 0:0.001:2
coeficientes3 = coef_fourier_cos(f,2,50)

In [ ]:
animacion = @animate for t ∈ 0:0.1:100
    ys = []
    for i ∈ 1:length(xs)
        y = solucion_cal_neumann(xs[i],t,coeficientes3,2)
        push!(ys,y)
    end
    plot(xs,ys,line_z=ys,linecolor=:thermal,linewidth=3,label=false,title ="t = $(t) s",
        ylims=(-0.1,1.02),xlims=(0,2),clim =(0,1))
    plot!(xs,-0.1*ones(length(xs)),line_z=ys,linecolor=:thermal,linewidth=40,label=false,dpi=150)
end

In [ ]:
gif(animacion,"sol_3.gif",fps =20)

In [ ]:
x2 = 0:0.001:4
h(x) = 4*x*(x-1)*(x-3)^2+7
coeficientes4 = coef_fourier_cos(h,4,50)

In [ ]:
animacion = @animate for t ∈ 0:0.1:100
    y2 = []
    for i ∈ 1:length(x2)
        y = solucion_cal_neumann(x2[i],t,coeficientes4,4)
        push!(y2,y)
    end
    plot(x2,y2,line_z=y2,linecolor=:thermal,linewidth=5,label=false,title="t = $(t) s",
        ylims=(-5,55),xlims=(0,4),clim =(7,55))
    plot!(x2,-5*ones(length(x2)),line_z=y2,linecolor=:thermal,linewidth=40,label=false,dpi=150)
end

In [ ]:
gif(animacion,"sol_4.gif",fps =20)

In [ ]:
function solucion_difusion(n,h,f,T₁,T₂,c;A=2,duracion=100)
    x=0:c*h:n*h*c
    X = spzeros(n+1,n+1)
    T = f.(x)
    b = zeros(n+1)
    Solucion = [T]
    for t ∈ 1:duracion
        for i ∈ 1:n+1
            if i==1
                X[i,i] = 1
                b[i] = T₁
            elseif i==n+1
                X[i,i] = 1
                b[i] = T₂
            else
                X[i,i-1] = A
                X[i,i+1] = A
                X[i,i] = -(2*A + h*c^2)
                b[i] = -h*T[i]*c^2
            end
        end
        T = X\b
        push!(Solucion,T)
    end
    return x, Solucion
end

In [ ]:
f(x) = -x^2 + 2*x
x, Sol = solucion_difusion(2000,0.1,f,0,0,0.01;A=0.01,duracion=1000)

In [ ]:
anim = @animate for i ∈ 1:length(Sol)
    plot(x,Sol[i],line_z=Sol[i],linecolor=:thermal,linewidth=3,label=false,title ="t = $(round(i*0.1, digits=1)) s",
        ylims=(-0.1,1.02),xlims=(0,2),clim =(0,1))
    plot!(x,-0.1*ones(length(x)),line_z=Sol[i],linecolor=:thermal,linewidth=40,label=false,dpi=150)
end

In [ ]:
gif(anim, "prueba1.gif", fps = 20)

In [ ]:
g(x) = 4*x*(x-1)*(x-3)^2+7
x2, Sol2 = solucion_difusion(4000,0.1,g,7,55,0.01;A=0.01,duracion=1000)

In [ ]:
anim = @animate for i ∈ 1:length(Sol2)
    plot(x2,Sol2[i],line_z=Sol2[i],linecolor=:thermal,linewidth=3,label=false,title ="t = $(round(i*0.1, digits=1)) s",
        ylims=(-5,55),xlims=(0,4),clim = (7,55))
    plot!(x2,-5*ones(length(x2)),line_z=Sol2[i],linecolor=:thermal,linewidth=40,label=false,dpi=150)
end

In [ ]:
gif(anim, "prueba2.gif", fps = 20)